In [3]:
import pandas as pd
import numpy as np

In [4]:
num = 54
df_final = pd.DataFrame(np.zeros((12, 58)))
# Read all 54 data files into python dataframe
for i in range(1,num+1):
    file = r"Fruit_Data/"+str(i)+"-ColorConceptAssocFr.xls"
    df = pd.read_excel(file, index_col=None, header = None, parse_cols = [x for x in range(3,16)])
    df = df.transpose()
    df = df.drop([0],axis = 1)
    df.columns = [x for x in range(0,58)]
    df = (df+200) / 400
    df_final = df_final.add(df)
# Average of the ratings per color per subject -- 12x58 matrix
df_final = df_final/num
df_final

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.111435,0.0762037,0.107685,0.108287,0.102315,0.115278,0.108889,0.111435,0.16,0.148611,...,0.38037,0.535231,0.751806,0.618102,0.670231,0.315833,0.262083,0.197176,0.599815,0.854213
1,0.0923148,0.067963,0.129815,0.11287,0.101528,0.185231,0.212083,0.252685,0.38338,0.400972,...,0.137824,0.116528,0.0924074,0.14338,0.128935,0.680972,0.268935,0.511296,0.112269,0.115139
2,0.121667,0.106204,0.128981,0.127037,0.0847685,0.0988426,0.136435,0.0934259,0.0986111,0.112083,...,0.297963,0.431667,0.297083,0.379398,0.220463,0.113889,0.375278,0.446574,0.324815,0.31588
3,0.092037,0.0944907,0.101944,0.0705556,0.094537,0.133472,0.151667,0.102593,0.129074,0.0943056,...,0.389722,0.540648,0.692824,0.780324,0.486574,0.215694,0.24412,0.266759,0.449444,0.725741
4,0.0917593,0.0835648,0.127222,0.115556,0.186111,0.230463,0.253333,0.230046,0.36537,0.297269,...,0.248519,0.365694,0.489861,0.578611,0.414676,0.325278,0.204398,0.125278,0.323194,0.454167
5,0.100787,0.0822685,0.0949074,0.1025,0.100833,0.218102,0.269815,0.318704,0.355139,0.417222,...,0.0978241,0.11963,0.118009,0.163657,0.175694,0.745694,0.151528,0.25625,0.108287,0.108565
6,0.126528,0.117685,0.106852,0.0936574,0.220833,0.3,0.282917,0.484259,0.430231,0.473889,...,0.104213,0.112546,0.0978704,0.190787,0.133333,0.559907,0.111019,0.130694,0.0935185,0.110139
7,0.512222,0.808056,0.596435,0.397917,0.278241,0.2175,0.129028,0.179722,0.136019,0.108565,...,0.10713,0.111806,0.113056,0.0869444,0.075,0.0653241,0.0979167,0.114398,0.0967593,0.0726389
8,0.111944,0.116806,0.101296,0.104398,0.100185,0.130926,0.13338,0.0665741,0.0716204,0.0787037,...,0.317454,0.208565,0.123287,0.119676,0.0818519,0.0654167,0.365556,0.529815,0.15037,0.11162
9,0.0971296,0.0783796,0.0805556,0.102731,0.0920833,0.100463,0.14,0.114259,0.121852,0.151343,...,0.281759,0.383565,0.849722,0.648009,0.859491,0.348981,0.170556,0.12125,0.424444,0.823194


In [5]:
from gurobipy import *
import math
import random
import time

In [6]:
## Define the ratings data and the desired ratings
r = {}
Fruits = [x for x in range(12)] # Later substitute with actual values -- maybe
Colors = [x for x in range(58)]
FC_Map = [(i,j) for i in Fruits for j in Colors]
FC_Map;

In [7]:
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.111435,0.0762037,0.107685,0.108287,0.102315,0.115278,0.108889,0.111435,0.16,0.148611,...,0.38037,0.535231,0.751806,0.618102,0.670231,0.315833,0.262083,0.197176,0.599815,0.854213
1,0.0923148,0.067963,0.129815,0.11287,0.101528,0.185231,0.212083,0.252685,0.38338,0.400972,...,0.137824,0.116528,0.0924074,0.14338,0.128935,0.680972,0.268935,0.511296,0.112269,0.115139
2,0.121667,0.106204,0.128981,0.127037,0.0847685,0.0988426,0.136435,0.0934259,0.0986111,0.112083,...,0.297963,0.431667,0.297083,0.379398,0.220463,0.113889,0.375278,0.446574,0.324815,0.31588
3,0.092037,0.0944907,0.101944,0.0705556,0.094537,0.133472,0.151667,0.102593,0.129074,0.0943056,...,0.389722,0.540648,0.692824,0.780324,0.486574,0.215694,0.24412,0.266759,0.449444,0.725741
4,0.0917593,0.0835648,0.127222,0.115556,0.186111,0.230463,0.253333,0.230046,0.36537,0.297269,...,0.248519,0.365694,0.489861,0.578611,0.414676,0.325278,0.204398,0.125278,0.323194,0.454167


In [9]:
# for i in Fruits:
#     r[i] = [df_final.loc[i][j] for j in Colors ]
for (i,j) in FC_Map:
    r[i,j] = df_final.loc[i,j]
    #print("Fruit",i,"Color",j," --> ",r[i,j])
k = 2             # Desired number of fruits k = 1,2,...p
l = 8             # Desired number of colors l = 1,2,...q
d = {}            # Specify the desired ratings for p fruits and q colors
for i in range(k):
    # d_(k,l) = Desired rate of association for the fruit 'k' with the color 'l'
    d[i] = [random.uniform(0,1) for j in range(l)]
d;

{0: [0.74217023662094,
  0.10370171857965349,
  0.5158462007341552,
  0.5716926813368829,
  0.6221164241375428,
  0.960549878188422,
  0.05712353819627669,
  0.7421475258203997],
 1: [0.5555446434558561,
  0.9721164231220115,
  0.7359963197736065,
  0.889553440963447,
  0.37431069008747064,
  0.30887502156637736,
  0.09626307865183703,
  0.44196838419308093]}